# PPE Equipment Detection - Webcam

In [ ]:
# Import necessary libraries
import cv2
from ultralytics import YOLO

# Load the trained YOLOv8 model
model = YOLO('model/best_version_3.pt')


In [ ]:
# Initialize the webcam
cap = cv2.VideoCapture(0)  # 0 is the default webcam


In [ ]:
# Function to process each frame
def process_frame(frame):
    # Reduce the size of the frame for faster processing
    resized_frame = cv2.resize(frame, (640, 480))
    
    # Run YOLOv8 inference on the frame
    results = model(resized_frame)
    
    # Draw the results on the frame
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # xyxy (N, 4)
        confs = result.boxes.conf.cpu().numpy()  # (N,)
        class_ids = result.boxes.cls.cpu().numpy()  # (N,)
        for box, conf, class_id in zip(boxes, confs, class_ids):
            x1, y1, x2, y2 = map(int, box)
            label = model.names[int(class_id)]  # Get the class name
            cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(resized_frame, f'{label} {conf:.2f}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    return resized_frame


In [ ]:
# Process the video and display the results
import time

try:
    frame_skip = 1  # Process every 2nd frame to reduce load
    frame_count = 0
    
    while True:
        # Capture frame-by-frame
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % frame_skip != 0:
            continue
        
        # Process the frame
        start_time = time.time()
        processed_frame = process_frame(frame)
        end_time = time.time()
        
        # Calculate and display the frame processing time
        fps = 1 / (end_time - start_time)
        cv2.putText(processed_frame, f'FPS: {fps:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # Display the resulting frame
        cv2.imshow('YOLOv8 Video Test', processed_frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Release the video capture and close windows
    cap.release()
    cv2.destroyAllWindows()


In [ ]:
# Release the webcam and close windows if not already done
cap.release()
cv2.destroyAllWindows()
